In [1]:
import useful as use
import pandas as pd
from pathlib import Path
from collections import defaultdict
import numpy as np

In [2]:
priv = use.get_priv()
global DATA_PATH
DATA_PATH = Path(priv["DATA_PATH"])

In [3]:
popular_df_raw = pd.read_pickle(DATA_PATH / "temp/raw/popular_raw.pkl")
unpopular_df_raw = pd.read_pickle(DATA_PATH/"temp/raw/unpopular_raw.pkl")

# Not used

In [11]:
def cleanse_tabular_data(popular_df_raw, unpopular_df_raw, dataset_dir):
    from Data_Processing import cleansing
    def cleanse_vid_df(df:pd.DataFrame, cleansing:dict):
        cleansed_df = pd.DataFrame([])
        cleansed_df["video_id"] = df["id"]
        cleansed_df["length_of_title"] = df["title"].apply(cleansing["length_of_title"])
        cleansed_df["length_of_description"] = df["description"].apply(cleansing["length_of_description"])
        cleansed_df["number_of_tags"] = df["tags"].apply(cleansing["number_of_tags"])
        def_lang_cleansed = df["default_language"].apply(cleansing["default_language"])
        def_audio_cleansed = df["default_audio_language"].apply(cleansing["default_audio_language"])
        cleansed_df["duration"] = df["duration"].apply(cleansing["duration"])
        cleansed_df["has_captions"] = df["has_captions"].apply(cleansing["has_captions"])
        cleansed_df["represents_licensed_content"] = df["represents_licensed_content"].apply(cleansing["represents_licensed_content"])
        cleansed_df["is_blocked_somewhere"] = df["blocked_region"].apply(cleansing["is_blocked_somewhere"])
        cleansed_df["views"] = df["views"].apply(cleansing["views"])
        likes = df["likes"].apply(cleansing["likes"])
        comments = df["comments"].apply(cleansing["comments"])
        cleansed_df["has_ppl"] = df["has_ppl"].apply(cleansing["has_ppl"])
        cleansed_df["was_streamed"] = df["stream_actual_start"].apply(cleansing["was_streamed"])
        cleansed_df = pd.concat([cleansed_df, def_lang_cleansed, def_audio_cleansed, likes, comments], axis=1)
        cleansed_df = cleansed_df.dropna() # Found an error regarding livestream video and duration, decided to drop rather than to make it some other value.
        return cleansed_df

    def label_dfs(df_list:list, labels:list, label_name="target") -> list:
        """
        Returns labeled dataframes with column name of `label_name`.
        ## Input
        - df_list: list of pandas.DataFrames
        - labels: corresponding list of label values
        - label_name: name of the label column, by default `target`
        ## Output
        - list of labeled pandas.DataFrames
        """
        # assert that the lengths of df_list and labels match up
        assert len(df_list) == len(labels)
        result = []
        for df, label in zip(df_list, labels):
            # assert that label_name should not already exist in the dataframes
            assert not df.get(label_name, False)
            df[label_name] = label
            result.append(df)
        return result
    def drop_vlc(df:pd.DataFrame):
        result = df.drop(["views", "likes", "private-likes", "comments", "private-comments"], axis=1)
        return result

    def construct_label_info(df:pd.DataFrame, target_name="target"):
        result = df[["video_id", target_name]].copy()
        return result


    popular_cleansed = cleanse_vid_df(popular_df_raw, cleansing)
    unpopular_cleansed = cleanse_vid_df(unpopular_df_raw, cleansing)

    popular_labeled, unpopular_labeled = label_dfs([popular_cleansed, unpopular_cleansed], [1, 0], "target")
    tabular_labeled = pd.concat([popular_labeled, unpopular_labeled]).reset_index(drop=True)
    tabular_labeled_no_vlc = drop_vlc(tabular_labeled)
    label_info = construct_label_info(tabular_labeled)
    
    use.create_directory_if_not_exists(dataset_dir)
    tabular_labeled.to_pickle(dataset_dir / "tabular_features_vlc.pkl")
    tabular_labeled_no_vlc.to_pickle(dataset_dir / "tabular_features_no_vlc.pkl")
    label_info.to_pickle(dataset_dir/"label_info.pkl")
    return tabular_labeled, tabular_labeled_no_vlc, label_info

tabular_features, tabular_features_no_vlc, label_info = cleanse_tabular_data(popular_df_raw, unpopular_df_raw, DATA_PATH / "dataset")

ModuleNotFoundError: No module named 'Data_Processing'

In [ ]:
def get_thumbnails(popular_df_raw, unpopular_df_raw, thumbnails_storage_dir):
    # Download thumbnails and store in thumbnails_storage_dir, with video id as file name
    popular_id_url = popular_df_raw[["id", "thumbnail_url"]]
    unpopular_id_url = unpopular_df_raw[["id", "thumbnail_url"]]

    def switch_to_hq(url_str):
        split_url = url_str.split("/")
        split_url[-1] = "hqdefault.jpg"
        rejoined_url = "/".join(split_url)
        return rejoined_url

    popular_id_url.loc[:, "thumbnail_url"] = popular_id_url["thumbnail_url"].apply(switch_to_hq)
    unpopular_id_url.loc[:, "thumbnail_url"] = unpopular_id_url["thumbnail_url"].apply(switch_to_hq)

    use.create_directory_if_not_exists(thumbnails_storage_dir)

    def download_image(progress_id:int, video_id:str, image_url:str, save_folder:Path):
        """
        Downloads an image from a URL and saves it to a specified folder.

        Args:
            image_url (str): The URL of the image to download.
            save_folder (str, optional): The folder to save the image in. Defaults to DATA_PATH / "popular_vids".
        """
        import requests
        try:
            response = requests.get(image_url, stream=True)
            response.raise_for_status()  # Raise an exception for bad status codes

            use.create_directory_if_not_exists(save_folder) # Create the folder if it doesn't exist

            filename = video_id + ".jpg"
            filepath = save_folder / filename

            with open(filepath, 'wb') as file:
                file.write(response.content)

            use.report_progress("DOWNLOAD", f"{filepath} --{progress_id}")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading {image_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {image_url}: {e}")
    for i, (video_id, image_url) in popular_id_url.iterrows():
        download_image(i, video_id, image_url, thumbnails_storage_dir / "1")
    for i, (video_id, image_url) in unpopular_id_url.iterrows():
        download_image(i, video_id, image_url, thumbnails_storage_dir / "0")
    return thumbnails_storage_dir / "1", thumbnails_storage_dir / "0"
    

In [ ]:
get_thumbnails(popular_df_raw, unpopular_df_raw, DATA_PATH/"dataset/thumbnails")

DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\eELPm0IUNtM.jpg --0
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\JSFG-IE8n_c.jpg --1
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\Z7S8_GraO9k.jpg --2
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\85DSFgZ4-wQ.jpg --3
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\7FW-U3ZOQM4.jpg --4
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\kQTFNQivQaE.jpg --5
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\kW1e8NtdW90.jpg --6
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\LsKcMV7hfPY.jpg --7
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\-XXPO8rl7Wk.jpg --8
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\iz3GdLbwHkI.jpg --9
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\SGP0pJZEYeM.jpg --10
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\hS0cRWkCRmw.jpg --11
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\-Ziv4u5c4G4.jpg --12
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\hHT8FNgW2NU.jpg --13
DOWNLOAD: G:\내 드라이브\Data\dataset\thumbnails\1\BAhjzJpecrQ.jpg --14
DOWNL

In [ ]:
popular_id_title = popular_df_raw[["id", "title"]]
unpopular_id_title = unpopular_df_raw[["id", "title"]]

In [ ]:
import shutil
import os

folder_path = DATA_PATH / "dataset/thumbnails"
output_path = DATA_PATH / "dataset/thumbnails.zip"
format_type = "zip"
try:
    # Create the directory for the output file if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Create the archive
    archive_path = shutil.make_archive(
        base_name=os.path.splitext(output_path)[0],  # Remove extension from output path for base_name
        format=format_type,
        root_dir=os.path.dirname(folder_path),     # The directory where the folder is located
        base_dir=os.path.basename(folder_path)      # The name of the folder to be archived
    )
    print(f"Successfully compressed '{folder_path}' to '{archive_path}'")
except FileNotFoundError:
    print(f"Error: Folder not found at '{folder_path}'")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully compressed 'G:\내 드라이브\Data\dataset\thumbnails' to 'G:\내 드라이브\Data\dataset\thumbnails.zip'


# Videos

In [4]:
features_dir = DATA_PATH / "dataset"
thumbnails_dir = DATA_PATH / "dataset/thumbnails"

In [6]:
import os
import json
import cv2
import numpy as np
from pathlib import Path
from yt_dlp import YoutubeDL

# Your DataFrame of video IDs
popular_id = popular_df_raw["id"]
unpopular_id = unpopular_df_raw["id"]

base_opts = {
    "format": "bestvideo[height<=360][ext=mp4][vcodec^=?avc]",  # Download best quality video up to 360p in MP4 format (no audio)
    "quiet": False,
}

# Function to calculate L1-norms between consecutive frames
def feature_from_video(video_path):
    cap = cv2.VideoCapture(str(video_path))
    l1_norms = []

    # Read first frame and convert to greyscale
    ret, prev_frame = cap.read()
    if not ret:
        print(f"Failed to read video: {video_path}")
        cap.release()
        return l1_norms

    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Process remaining frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to greyscale
        curr_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate L1-norm (sum of absolute differences)
        l1_norm = np.sum(np.abs(curr_frame_gray - prev_frame_gray))
        l1_norms.append(l1_norm)

        # Update previous frame
        prev_frame_gray = curr_frame_gray

    cap.release()
    return l1_norms

def convert_numpy(obj):
    if isinstance(obj, np.generic):
        return obj.item()
    raise TypeError(f"Type {type(obj)} not serializable")

# Dictionary to store L1-norms for each video
video_l1_norms = {}

# Ensure the directory exists
output_path = features_dir/"videos"
use.create_directory_if_not_exists(output_path)

total = len(popular_id)
# Download and process videos
for i, video_id in enumerate(popular_id):
    print(f"{i+1}/{total}")
    url = f"https://www.youtube.com/watch?v={video_id}"
    ydl_opts = base_opts.copy()
    video_file = output_path / f"{video_id}.mp4"
    ydl_opts["outtmpl"] = str(video_file)

    with YoutubeDL(ydl_opts) as ydl:
        try:
            print(f"Downloading {video_id}...")
            ydl.download([url])

            if os.path.exists(video_file):
                print(f"Successfully downloaded {video_id}")
            else:
                print(f"Failed to download {video_id}")
                continue # Skip to the next video if download failed

            # Check if the downloaded file exists and is an mp4 before processing
            if os.path.exists(video_file) and video_file.suffix == ".mp4":
                print(f"Processing video: {video_id}...")
                l1_norms = feature_from_video(video_file)

                # Store the results in the dictionary
                video_l1_norms[video_id] = l1_norms

                # Delete the video after processing
                os.remove(video_file)
                print(f"Deleted {video_id}.mp4")

                # Save the current state to a JSON file after processing each video
                with open(features_dir / "popular_l1_norms.json", "w") as json_file:
                    json.dump(video_l1_norms, json_file, default=convert_numpy)
            else:
                print(f"Video file not found or not an MP4 for {video_id} after download.")

        except Exception as e:
            print(f"Failed to process {video_id}: {e}")
            continue  # Skip this video and move to the next


1/308
[youtube] Extracting URL: https://www.youtube.com/watch?v=eELPm0IUNtM
[youtube] eELPm0IUNtM: Downloading webpage
[youtube] eELPm0IUNtM: Downloading tv client config
[youtube] eELPm0IUNtM: Downloading player 6450230e-main
[youtube] eELPm0IUNtM: Downloading tv player API JSON
[youtube] eELPm0IUNtM: Downloading ios player API JSON
[youtube] eELPm0IUNtM: Downloading m3u8 information
[info] eELPm0IUNtM: Downloading 1 format(s): 134
[download] Destination: /mnt/g/My Drive/Data/dataset/videos/eELPm0IUNtM.mp4
[download] 100% of   65.42MiB in -1:59:54 at N/A/s         
Successfully downloaded eELPm0IUNtM
Processing video: eELPm0IUNtM...
Deleted eELPm0IUNtM.mp4
2/308
[youtube] Extracting URL: https://www.youtube.com/watch?v=JSFG-IE8n_c
[youtube] JSFG-IE8n_c: Downloading webpage
[youtube] JSFG-IE8n_c: Downloading tv client config
[youtube] JSFG-IE8n_c: Downloading player 6450230e-main
[youtube] JSFG-IE8n_c: Downloading tv player API JSON
[youtube] JSFG-IE8n_c: Downloading ios player API JSO

ERROR: [youtube] Pw_IenodtsM: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Failed to process Pw_IenodtsM: ERROR: [youtube] Pw_IenodtsM: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
241/308
[youtube] Extracting URL: https://www.youtube.com/watch?v=BgFbfqBSX_8
[youtube] BgFbfqBSX_8: Downloading webpage
[youtube] BgFbfqBSX_8: Downloading tv client config
[youtube] BgFbfqBSX_8: Downloading player 6450230e-main
[youtube] BgFbfqBSX_8: Downloading tv player API JSON
[youtube] BgFbfqBSX_8: Downloading ios player API JSON
[youtube] BgFbfqBSX_8: Downloading m3u8 information
[info] BgFbfqBSX_8: Downloading 1 format(s): 134
[download] Destination: /mnt/g/My Drive/Data/dataset/videos/BgFbfqBSX_8.mp4
[download] 100% of    1.22MiB in 00:00:00 

ERROR: [youtube] cNcl3yomZgM: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Failed to process cNcl3yomZgM: ERROR: [youtube] cNcl3yomZgM: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
262/308
[youtube] Extracting URL: https://www.youtube.com/watch?v=nD2Bxpm7_vg
[youtube] nD2Bxpm7_vg: Downloading webpage
[youtube] nD2Bxpm7_vg: Downloading tv client config
[youtube] nD2Bxpm7_vg: Downloading player 6450230e-main
[youtube] nD2Bxpm7_vg: Downloading tv player API JSON
[youtube] nD2Bxpm7_vg: Downloading ios player API JSON
[youtube] nD2Bxpm7_vg: Downloading m3u8 information
[info] nD2Bxpm7_vg: Downloading 1 format(s): 134
[download] Destination: /mnt/g/My Drive/Data/dataset/videos/nD2Bxpm7_vg.mp4
[download] 100% of    9.69MiB in 00:00:01 

In [8]:
with open(features_dir / "popular_l1_norms.json", "r") as json_file:
    popular_l1_norms = json.load(json_file)
len(popular_l1_norms)

306

In [ ]:
# Ensure the directory exists
output_path = features_dir/"videos"
use.create_directory_if_not_exists(output_path)

total = len(unpopular_id)
# Download and process videos
for i, video_id in enumerate(unpopular_id):
    print(f"{i+1}/{total}")
    url = f"https://www.youtube.com/watch?v={video_id}"
    ydl_opts = base_opts.copy()
    video_file = output_path / f"{video_id}.mp4"
    ydl_opts["outtmpl"] = str(video_file)

    with YoutubeDL(ydl_opts) as ydl:
        try:
            print(f"Downloading {video_id}...")
            ydl.download([url])

            if os.path.exists(video_file):
                print(f"Successfully downloaded {video_id}")
            else:
                print(f"Failed to download {video_id}")
                continue # Skip to the next video if download failed

            # Check if the downloaded file exists and is an mp4 before processing
            if os.path.exists(video_file) and video_file.suffix == ".mp4":
                print(f"Processing video: {video_id}...")
                l1_norms = feature_from_video(video_file)

                # Store the results in the dictionary
                video_l1_norms[video_id] = l1_norms

                # Delete the video after processing
                os.remove(video_file)
                print(f"Deleted {video_id}.mp4")

                # Save the current state to a JSON file after processing each video
                with open(features_dir / "unpopular_l1_norms.json", "w") as json_file:
                    json.dump(video_l1_norms, json_file, default=convert_numpy)
            else:
                print(f"Video file not found or not an MP4 for {video_id} after download.")

        except Exception as e:
            print(f"Failed to process {video_id}: {e}")
            continue  # Skip this video and move to the next

1/12367
[youtube] Extracting URL: https://www.youtube.com/watch?v=1joVcP50_zw
[youtube] 1joVcP50_zw: Downloading webpage
[youtube] 1joVcP50_zw: Downloading tv client config
[youtube] 1joVcP50_zw: Downloading player 6450230e-main
[youtube] 1joVcP50_zw: Downloading tv player API JSON
[youtube] 1joVcP50_zw: Downloading ios player API JSON
[youtube] 1joVcP50_zw: Downloading m3u8 information
[info] 1joVcP50_zw: Downloading 1 format(s): 160
[download] Destination: /mnt/g/My Drive/Data/dataset/videos/1joVcP50_zw.mp4
[download] 100% of  499.70KiB in 00:00:01 at 307.32KiB/s 
Successfully downloaded 1joVcP50_zw
Processing video: 1joVcP50_zw...
Deleted 1joVcP50_zw.mp4
2/12367
[youtube] Extracting URL: https://www.youtube.com/watch?v=8XPNGOERAdQ
[youtube] 8XPNGOERAdQ: Downloading webpage
[youtube] 8XPNGOERAdQ: Downloading tv client config
[youtube] 8XPNGOERAdQ: Downloading player 6450230e-main
[youtube] 8XPNGOERAdQ: Downloading tv player API JSON
[youtube] 8XPNGOERAdQ: Downloading ios player API J

ERROR: [youtube] mYvSVicz0tE: Video unavailable. The uploader has not made this video available in your country


Failed to process mYvSVicz0tE: ERROR: [youtube] mYvSVicz0tE: Video unavailable. The uploader has not made this video available in your country
20/12367
[youtube] Extracting URL: https://www.youtube.com/watch?v=jo2C6r5QXNo
[youtube] jo2C6r5QXNo: Downloading webpage
[youtube] jo2C6r5QXNo: Downloading tv client config
[youtube] jo2C6r5QXNo: Downloading player 6450230e-main
[youtube] jo2C6r5QXNo: Downloading tv player API JSON
[youtube] jo2C6r5QXNo: Downloading ios player API JSON


ERROR: [youtube] jo2C6r5QXNo: Video unavailable. The uploader has not made this video available in your country


Failed to process jo2C6r5QXNo: ERROR: [youtube] jo2C6r5QXNo: Video unavailable. The uploader has not made this video available in your country
21/12367
[youtube] Extracting URL: https://www.youtube.com/watch?v=8ogKNuuCfF0
[youtube] 8ogKNuuCfF0: Downloading webpage
[youtube] 8ogKNuuCfF0: Downloading tv client config
[youtube] 8ogKNuuCfF0: Downloading player 6450230e-main
[youtube] 8ogKNuuCfF0: Downloading tv player API JSON
[youtube] 8ogKNuuCfF0: Downloading ios player API JSON


...

Failed to process hL2Y3sXdR4w: ERROR: [youtube] hL2Y3sXdR4w: Video unavailable. The uploader has not made this video available in your country
34/12367
[youtube] Extracting URL: https://www.youtube.com/watch?v=75peTuePnHM
[youtube] 75peTuePnHM: Downloading webpage
[youtube] 75peTuePnHM: Downloading tv client config
[youtube] 75peTuePnHM: Downloading player 6450230e-main
[youtube] 75peTuePnHM: Downloading tv player API JSON
[youtube] 75peTuePnHM: Downloading ios player API JSON


ERROR: [youtube] 75peTuePnHM: Video unavailable. The uploader has not made this video available in your country


Failed to process 75peTuePnHM: ERROR: [youtube] 75peTuePnHM: Video unavailable. The uploader has not made this video available in your country
35/12367
[youtube] Extracting URL: https://www.youtube.com/watch?v=xfAUbKzgZPs
[youtube] xfAUbKzgZPs: Downloading webpage
[youtube] xfAUbKzgZPs: Downloading tv client config
[youtube] xfAUbKzgZPs: Downloading player 6450230e-main
[youtube] xfAUbKzgZPs: Downloading tv player API JSON
[youtube] xfAUbKzgZPs: Downloading ios player API JSON


ERROR: [youtube] xfAUbKzgZPs: Video unavailable. The uploader has not made this video available in your country


Failed to process xfAUbKzgZPs: ERROR: [youtube] xfAUbKzgZPs: Video unavailable. The uploader has not made this video available in your country
36/12367
[youtube] Extracting URL: https://www.youtube.com/watch?v=iY0PahNALQs
[youtube] iY0PahNALQs: Downloading webpage
[youtube] iY0PahNALQs: Downloading tv client config
[youtube] iY0PahNALQs: Downloading player 6450230e-main
[youtube] iY0PahNALQs: Downloading tv player API JSON
[youtube] iY0PahNALQs: Downloading ios player API JSON
[youtube] iY0PahNALQs: Downloading m3u8 information
[info] iY0PahNALQs: Downloading 1 format(s): 160
[download] Destination: /mnt/g/My Drive/Data/dataset/videos/iY0PahNALQs.mp4
[download] 100% of   67.20KiB in 00:00:00 at 214.78KiB/s 
Successfully downloaded iY0PahNALQs
Processing video: iY0PahNALQs...
Deleted iY0PahNALQs.mp4
37/12367
[youtube] Extracting URL: https://www.youtube.com/watch?v=pZasE4pcfRs
[youtube] pZasE4pcfRs: Downloading webpage
[youtube] pZasE4pcfRs: Downloading tv client config
[youtube] pZasE4